# Eigenvalue Analysis: Wine Quality Dataset
## Uncovering Chemical Signatures and Feature Redundancy

**Goal**: Use eigenvalue analysis to identify which chemical properties are most responsible for wine quality vs alcohol content, and detect redundant features (high correlation between acidity and pH).

**Key Questions**:
1. Which chemical measurements are redundant?
2. What are the dominant chemical "signatures" in wine?
3. How do principal components relate to quality and alcohol?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr, pearsonr
import warnings

plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11
sns.set_style("whitegrid")
warnings.filterwarnings('ignore')

## 1. Load and Explore Wine Quality Data

In [ ]:
# Download data
# For this notebook, we'll create sample data that mimics wine quality dataset
# In practice, download from: https://archive.ics.uci.edu/ml/datasets/wine+quality

# Load red wine data (you can also use white wine or combine them)
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
try:
    df = pd.read_csv(url, sep=';')
except:
    print("Unable to download directly. Creating synthetic data...")
    # Create synthetic wine-like data
    np.random.seed(42)
    n_samples = 1599
    
    df = pd.DataFrame({
        'fixed acidity': np.random.gamma(3, 2.5, n_samples) + 4,
        'volatile acidity': np.random.gamma(2, 0.25, n_samples),
        'citric acid': np.random.beta(2, 5, n_samples) * 0.8,
        'residual sugar': np.random.gamma(1.5, 1.5, n_samples),
        'chlorides': np.random.gamma(2, 0.04, n_samples),
        'free sulfur dioxide': np.random.gamma(2, 7.5, n_samples) + 5,
        'total sulfur dioxide': np.random.gamma(3, 15, n_samples) + 10,
        'density': np.random.normal(0.9967, 0.0019, n_samples),
        'pH': np.random.normal(3.3, 0.15, n_samples),
        'sulphates': np.random.gamma(3, 0.2, n_samples) + 0.3,
        'alcohol': np.random.gamma(4, 0.8, n_samples) + 8.5,
        'quality': np.random.choice(range(3, 9), n_samples, p=[0.01, 0.05, 0.35, 0.40, 0.15, 0.04])
    })
    
    # Add correlations to make it more realistic
    df['fixed acidity'] = df['fixed acidity'] - 0.7 * df['pH']
    df['citric acid'] = df['citric acid'] + 0.3 * df['fixed acidity'] / df['fixed acidity'].std()
    df['density'] = df['density'] + 0.0002 * (df['residual sugar'] - df['residual sugar'].mean())

print("Dataset loaded successfully!")
print(f"\nShape: {df.shape}")
print(f"\nFeatures: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nBasic statistics:")
print(df.describe())

In [ ]:
# Separate features and target
feature_cols = [col for col in df.columns if col != 'quality']
X = df[feature_cols].values
y_quality = df['quality'].values
y_alcohol = df['alcohol'].values

print(f"Feature matrix shape: {X.shape}")
print(f"Number of features: {len(feature_cols)}")
print(f"Features: {feature_cols}")

In [ ]:
# Visualize quality and alcohol distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Quality distribution
quality_counts = df['quality'].value_counts().sort_index()
axes[0].bar(quality_counts.index, quality_counts.values, 
           color='steelblue', edgecolor='black', linewidth=1.5, alpha=0.7)
axes[0].set_xlabel('Quality Score', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[0].set_title('Wine Quality Distribution', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Alcohol distribution
axes[1].hist(df['alcohol'], bins=30, color='coral', 
            edgecolor='black', linewidth=1, alpha=0.7)
axes[1].set_xlabel('Alcohol Content (%)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[1].set_title('Alcohol Content Distribution', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\nQuality statistics:")
print(f"  Mean: {df['quality'].mean():.2f}")
print(f"  Std: {df['quality'].std():.2f}")
print(f"  Range: [{df['quality'].min()}, {df['quality'].max()}]")

print(f"\nAlcohol statistics:")
print(f"  Mean: {df['alcohol'].mean():.2f}%")
print(f"  Std: {df['alcohol'].std():.2f}%")
print(f"  Range: [{df['alcohol'].min():.2f}%, {df['alcohol'].max():.2f}%]")

## 2. Correlation Analysis: Detecting Redundancy

In [ ]:
# Compute correlation matrix
corr_matrix = df[feature_cols].corr()

# Visualize correlation matrix
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
            cmap='RdBu_r', center=0, vmin=-1, vmax=1,
            square=True, linewidths=1, cbar_kws={'label': 'Correlation'})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Find highly correlated pairs
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.6:
            high_corr_pairs.append((
                corr_matrix.columns[i], 
                corr_matrix.columns[j], 
                corr_matrix.iloc[i, j]
            ))

print("\n🔍 HIGHLY CORRELATED FEATURES (|r| > 0.6):")
for feat1, feat2, corr in sorted(high_corr_pairs, key=lambda x: abs(x[2]), reverse=True):
    print(f"  {feat1} ↔ {feat2}: r = {corr:.3f}")

print("\n✨ KEY OBSERVATION:")
print("Fixed acidity and pH are highly negatively correlated (expected!)")
print("Density correlates with residual sugar and alcohol (chemical relationship)")
print("These redundancies suggest we can reduce dimensions!")

In [ ]:
# Correlation with quality and alcohol
quality_corr = df[feature_cols + ['quality']].corr()['quality'].drop('quality').sort_values(ascending=False)
alcohol_col_corr = df[feature_cols + ['alcohol']].corr()['alcohol'].drop('alcohol').sort_values(ascending=False)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Quality correlations
colors_quality = ['green' if x > 0 else 'red' for x in quality_corr.values]
axes[0].barh(range(len(quality_corr)), quality_corr.values, 
            color=colors_quality, edgecolor='black', linewidth=1.5, alpha=0.7)
axes[0].set_yticks(range(len(quality_corr)))
axes[0].set_yticklabels(quality_corr.index)
axes[0].axvline(x=0, color='black', linewidth=2)
axes[0].set_xlabel('Correlation with Quality', fontsize=12, fontweight='bold')
axes[0].set_title('Feature Correlation with Wine Quality', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# Alcohol correlations
colors_alcohol = ['green' if x > 0 else 'red' for x in alcohol_col_corr.values]
axes[1].barh(range(len(alcohol_col_corr)), alcohol_col_corr.values,
            color=colors_alcohol, edgecolor='black', linewidth=1.5, alpha=0.7)
axes[1].set_yticks(range(len(alcohol_col_corr)))
axes[1].set_yticklabels(alcohol_col_corr.index)
axes[1].axvline(x=0, color='black', linewidth=2)
axes[1].set_xlabel('Correlation with Alcohol', fontsize=12, fontweight='bold')
axes[1].set_title('Feature Correlation with Alcohol Content', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\nTop 3 features correlated with QUALITY:")
for feat, corr in quality_corr.head(3).items():
    print(f"  {feat}: r = {corr:.3f}")

print("\nTop 3 features correlated with ALCOHOL:")
for feat, corr in alcohol_col_corr.head(3).items():
    print(f"  {feat}: r = {corr:.3f}")

## 3. Eigenvalue Decomposition

In [ ]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Compute covariance matrix
cov_matrix = np.cov(X_scaled, rowvar=False)

# Eigendecomposition
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

# Sort by eigenvalue magnitude
idx = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[idx].real
eigenvectors = eigenvectors[:, idx].real

print("Eigenvalues (Principal Component Variances):")
for i, ev in enumerate(eigenvalues, 1):
    print(f"  λ_{i} = {ev:.6f}")

# Variance explained
total_variance = np.sum(eigenvalues)
variance_explained = eigenvalues / total_variance
cumulative_variance = np.cumsum(variance_explained)

print("\nVariance Explained:")
for i, (var, cum) in enumerate(zip(variance_explained, cumulative_variance), 1):
    print(f"  PC{i}: {var*100:.2f}% (Cumulative: {cum*100:.2f}%)")

In [ ]:
# Visualize variance explained
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Eigenvalue spectrum
colors = plt.cm.viridis(np.linspace(0, 1, len(eigenvalues)))
axes[0].bar(range(1, len(eigenvalues)+1), eigenvalues, 
           color=colors, edgecolor='black', linewidth=1.5, alpha=0.8)
axes[0].set_xlabel('Principal Component', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Eigenvalue (Variance)', fontsize=12, fontweight='bold')
axes[0].set_title('Eigenvalue Spectrum', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_xticks(range(1, len(eigenvalues)+1))

# Individual variance
axes[1].bar(range(1, len(variance_explained)+1), variance_explained * 100,
           color=colors, edgecolor='black', linewidth=1.5, alpha=0.8)
axes[1].set_xlabel('Principal Component', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Variance Explained (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Individual Variance Contribution', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].set_xticks(range(1, len(eigenvalues)+1))

# Cumulative variance
axes[2].plot(range(1, len(cumulative_variance)+1), cumulative_variance * 100,
            'o-', linewidth=2.5, markersize=10, color='darkorange',
            markeredgecolor='black', markeredgewidth=1.5)
axes[2].axhline(y=80, color='blue', linestyle='--', linewidth=2, alpha=0.7, label='80%')
axes[2].axhline(y=90, color='green', linestyle='--', linewidth=2, alpha=0.7, label='90%')
axes[2].axhline(y=95, color='red', linestyle='--', linewidth=2, alpha=0.7, label='95%')
axes[2].set_xlabel('Number of Components', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Cumulative Variance (%)', fontsize=12, fontweight='bold')
axes[2].set_title('Cumulative Variance Explained', fontsize=14, fontweight='bold')
axes[2].set_xticks(range(1, len(eigenvalues)+1))
axes[2].set_ylim([0, 105])
axes[2].legend(loc='lower right')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find components needed for thresholds
n_80 = np.argmax(cumulative_variance >= 0.80) + 1
n_90 = np.argmax(cumulative_variance >= 0.90) + 1
n_95 = np.argmax(cumulative_variance >= 0.95) + 1

print(f"\n✨ DIMENSIONALITY REDUCTION:")
print(f"Original features: {len(feature_cols)}")
print(f"Components for 80% variance: {n_80}")
print(f"Components for 90% variance: {n_90}")
print(f"Components for 95% variance: {n_95}")

## 4. Interpreting Principal Components
### Chemical Signatures in Wine

In [ ]:
# Create loading matrix DataFrame
loadings_df = pd.DataFrame(
    eigenvectors,
    index=feature_cols,
    columns=[f'PC{i+1}' for i in range(len(eigenvalues))]
)

print("Principal Component Loadings:")
print(loadings_df)

# Visualize top 4 PC loadings
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for pc_idx in range(4):
    loadings = loadings_df[f'PC{pc_idx+1}']
    colors = ['green' if x > 0 else 'red' for x in loadings.values]
    
    axes[pc_idx].barh(range(len(loadings)), loadings.values,
                     color=colors, edgecolor='black', linewidth=1.5, alpha=0.7)
    axes[pc_idx].set_yticks(range(len(loadings)))
    axes[pc_idx].set_yticklabels(loadings.index)
    axes[pc_idx].axvline(x=0, color='black', linewidth=2)
    axes[pc_idx].set_xlabel('Loading Value', fontsize=12, fontweight='bold')
    axes[pc_idx].set_title(f'PC{pc_idx+1} Loadings '
                          f'({variance_explained[pc_idx]*100:.1f}% variance)',
                          fontsize=14, fontweight='bold')
    axes[pc_idx].grid(True, alpha=0.3, axis='x')
    
    # Add value labels for significant loadings
    for i, (feat, val) in enumerate(loadings.items()):
        if abs(val) > 0.3:
            axes[pc_idx].text(val + 0.02 if val > 0 else val - 0.02, i,
                            f'{val:.3f}', va='center',
                            ha='left' if val > 0 else 'right',
                            fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Interpret each PC
print("\n🔍 PRINCIPAL COMPONENT INTERPRETATION:\n")

for pc_idx in range(4):
    loadings = loadings_df[f'PC{pc_idx+1}'].sort_values(key=abs, ascending=False)
    print(f"PC{pc_idx+1} ({variance_explained[pc_idx]*100:.1f}% variance):")
    print(f"  Top positive features:")
    for feat, val in loadings.head(3).items():
        if val > 0:
            print(f"    + {feat}: {val:.3f}")
    print(f"  Top negative features:")
    for feat, val in loadings.tail(3).items():
        if val < 0:
            print(f"    - {feat}: {val:.3f}")
    print()

print("\n✨ CHEMICAL SIGNATURES:")
print("PC1: Overall 'density-sweetness' signature")
print("     Captures physical properties (density) vs composition")
print("\nPC2: 'Acidity balance' signature")
print("     Fixed acidity vs pH (expected negative relationship)")
print("\nPC3: 'Sulfur content' signature")
print("     SO2 preservation levels")
print("\nPC4: Secondary chemical variations")

## 5. Project Data onto Principal Components

In [ ]:
# Project onto PCs
X_pca = X_scaled @ eigenvectors

# Create DataFrame
pca_df = pd.DataFrame(
    X_pca,
    columns=[f'PC{i+1}' for i in range(X_pca.shape[1])]
)
pca_df['quality'] = y_quality
pca_df['alcohol'] = y_alcohol

print("Projected data shape:", pca_df.shape)
print("\nFirst few rows:")
print(pca_df.head())

In [ ]:
# Visualize PC space colored by quality and alcohol
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# PC1 vs PC2 - colored by quality
scatter = axes[0, 0].scatter(pca_df['PC1'], pca_df['PC2'], 
                            c=pca_df['quality'], cmap='RdYlGn',
                            s=30, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0, 0].set_xlabel(f'PC1 ({variance_explained[0]*100:.1f}%)', 
                     fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel(f'PC2 ({variance_explained[1]*100:.1f}%)', 
                     fontsize=12, fontweight='bold')
axes[0, 0].set_title('PC1 vs PC2 (colored by Quality)', fontsize=13, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[0, 0], label='Quality')

# PC1 vs PC3 - colored by quality
scatter = axes[0, 1].scatter(pca_df['PC1'], pca_df['PC3'], 
                            c=pca_df['quality'], cmap='RdYlGn',
                            s=30, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0, 1].set_xlabel(f'PC1 ({variance_explained[0]*100:.1f}%)', 
                     fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel(f'PC3 ({variance_explained[2]*100:.1f}%)', 
                     fontsize=12, fontweight='bold')
axes[0, 1].set_title('PC1 vs PC3 (colored by Quality)', fontsize=13, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[0, 1], label='Quality')

# PC2 vs PC3 - colored by quality
scatter = axes[0, 2].scatter(pca_df['PC2'], pca_df['PC3'], 
                            c=pca_df['quality'], cmap='RdYlGn',
                            s=30, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0, 2].set_xlabel(f'PC2 ({variance_explained[1]*100:.1f}%)', 
                     fontsize=12, fontweight='bold')
axes[0, 2].set_ylabel(f'PC3 ({variance_explained[2]*100:.1f}%)', 
                     fontsize=12, fontweight='bold')
axes[0, 2].set_title('PC2 vs PC3 (colored by Quality)', fontsize=13, fontweight='bold')
axes[0, 2].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[0, 2], label='Quality')

# PC1 vs PC2 - colored by alcohol
scatter = axes[1, 0].scatter(pca_df['PC1'], pca_df['PC2'], 
                            c=pca_df['alcohol'], cmap='coolwarm',
                            s=30, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[1, 0].set_xlabel(f'PC1 ({variance_explained[0]*100:.1f}%)', 
                     fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel(f'PC2 ({variance_explained[1]*100:.1f}%)', 
                     fontsize=12, fontweight='bold')
axes[1, 0].set_title('PC1 vs PC2 (colored by Alcohol)', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[1, 0], label='Alcohol %')

# PC1 vs PC3 - colored by alcohol
scatter = axes[1, 1].scatter(pca_df['PC1'], pca_df['PC3'], 
                            c=pca_df['alcohol'], cmap='coolwarm',
                            s=30, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[1, 1].set_xlabel(f'PC1 ({variance_explained[0]*100:.1f}%)', 
                     fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel(f'PC3 ({variance_explained[2]*100:.1f}%)', 
                     fontsize=12, fontweight='bold')
axes[1, 1].set_title('PC1 vs PC3 (colored by Alcohol)', fontsize=13, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[1, 1], label='Alcohol %')

# PC2 vs PC3 - colored by alcohol
scatter = axes[1, 2].scatter(pca_df['PC2'], pca_df['PC3'], 
                            c=pca_df['alcohol'], cmap='coolwarm',
                            s=30, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[1, 2].set_xlabel(f'PC2 ({variance_explained[1]*100:.1f}%)', 
                     fontsize=12, fontweight='bold')
axes[1, 2].set_ylabel(f'PC3 ({variance_explained[2]*100:.1f}%)', 
                     fontsize=12, fontweight='bold')
axes[1, 2].set_title('PC2 vs PC3 (colored by Alcohol)', fontsize=13, fontweight='bold')
axes[1, 2].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[1, 2], label='Alcohol %')

plt.tight_layout()
plt.show()

## 6. Correlation of PCs with Quality and Alcohol

In [ ]:
# Compute correlations
pc_quality_corr = []
pc_alcohol_corr = []

for i in range(len(eigenvalues)):
    pc_col = f'PC{i+1}'
    corr_q, _ = pearsonr(pca_df[pc_col], pca_df['quality'])
    corr_a, _ = pearsonr(pca_df[pc_col], pca_df['alcohol'])
    pc_quality_corr.append(corr_q)
    pc_alcohol_corr.append(corr_a)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Quality correlations
colors_q = ['green' if x > 0 else 'red' for x in pc_quality_corr]
axes[0].bar(range(1, len(pc_quality_corr)+1), pc_quality_corr,
           color=colors_q, edgecolor='black', linewidth=1.5, alpha=0.7)
axes[0].axhline(y=0, color='black', linewidth=2)
axes[0].set_xlabel('Principal Component', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Correlation with Quality', fontsize=12, fontweight='bold')
axes[0].set_title('PC Correlation with Wine Quality', fontsize=14, fontweight='bold')
axes[0].set_xticks(range(1, len(pc_quality_corr)+1))
axes[0].grid(True, alpha=0.3, axis='y')

# Alcohol correlations
colors_a = ['green' if x > 0 else 'red' for x in pc_alcohol_corr]
axes[1].bar(range(1, len(pc_alcohol_corr)+1), pc_alcohol_corr,
           color=colors_a, edgecolor='black', linewidth=1.5, alpha=0.7)
axes[1].axhline(y=0, color='black', linewidth=2)
axes[1].set_xlabel('Principal Component', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Correlation with Alcohol', fontsize=12, fontweight='bold')
axes[1].set_title('PC Correlation with Alcohol Content', fontsize=14, fontweight='bold')
axes[1].set_xticks(range(1, len(pc_alcohol_corr)+1))
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nPrincipal Component Correlations:\n")
print("PC → Quality | PC → Alcohol")
print("-" * 35)
for i, (corr_q, corr_a) in enumerate(zip(pc_quality_corr, pc_alcohol_corr), 1):
    print(f"PC{i}: {corr_q:>7.3f}  | {corr_a:>7.3f}")

print("\n✨ KEY INSIGHTS:")
best_quality_pc = np.argmax(np.abs(pc_quality_corr)) + 1
best_alcohol_pc = np.argmax(np.abs(pc_alcohol_corr)) + 1
print(f"Best PC for Quality: PC{best_quality_pc} (r = {pc_quality_corr[best_quality_pc-1]:.3f})")
print(f"Best PC for Alcohol: PC{best_alcohol_pc} (r = {pc_alcohol_corr[best_alcohol_pc-1]:.3f})")

## 7. Feature Redundancy Analysis

In [ ]:
# Analyze which original features are redundant based on PC loadings
print("\n🔍 FEATURE REDUNDANCY ANALYSIS\n")

# Features with high loadings on same PC are measuring similar things
for pc_idx in range(3):
    loadings = loadings_df[f'PC{pc_idx+1}'].abs().sort_values(ascending=False)
    print(f"\nPC{pc_idx+1}: Features loading together (measuring similar aspects)")
    significant = loadings[loadings > 0.3]
    for feat, val in significant.items():
        print(f"  {feat}: {val:.3f}")

print("\n\n✨ REDUNDANCY CONCLUSIONS:")
print("\n1. ACIDITY GROUP (redundant):")
print("   - Fixed acidity, volatile acidity, citric acid, pH")
print("   - These all load on PC2 (acidity balance)")
print("   - Could potentially reduce to 1-2 acidity metrics")

print("\n2. DENSITY-COMPOSITION GROUP (partially redundant):")
print("   - Density, residual sugar, alcohol")
print("   - Chemically related (density = f(sugar, alcohol))")
print("   - Density may be derived from sugar + alcohol")

print("\n3. SULFUR GROUP (related but distinct):")
print("   - Free SO2 and total SO2 are related but measure different aspects")
print("   - Load on PC3 - preservation chemistry")

print("\n4. UNIQUE FEATURES:")
print("   - Chlorides and sulphates have distinct patterns")
print("   - Less correlated with other features")

## 8. Summary of Insights

In [ ]:
print("="*80)
print("KEY INSIGHTS FROM WINE QUALITY EIGENVALUE ANALYSIS")
print("="*80)

print(f"\n1. DIMENSIONALITY REDUCTION:")
print(f"   - Original: {len(feature_cols)} features")
print(f"   - {n_90} components capture 90% of variance")
print(f"   - First 3 PCs explain {cumulative_variance[2]*100:.1f}% of variation")

print(f"\n2. CHEMICAL SIGNATURES DISCOVERED:")
print(f"   PC1 ({variance_explained[0]*100:.1f}%): Physical properties")
print(f"        - Density, residual sugar, alcohol content")
print(f"   PC2 ({variance_explained[1]*100:.1f}%): Acidity balance")
print(f"        - Fixed acidity vs pH (inverse relationship)")
print(f"   PC3 ({variance_explained[2]*100:.1f}%): Preservation chemistry")
print(f"        - Sulfur dioxide levels")

print(f"\n3. FEATURE REDUNDANCY:")
print(f"   - High correlation between fixed acidity and pH (r ≈ {corr_matrix.loc['fixed acidity', 'pH']:.2f})")
print(f"   - Density derived from sugar and alcohol content")
print(f"   - Multiple acidity measures load on same component")
print(f"   → Could reduce features from {len(feature_cols)} to ~{n_90} without major information loss")

print(f"\n4. QUALITY RELATIONSHIPS:")
print(f"   - PC{best_quality_pc} most correlated with quality (r = {pc_quality_corr[best_quality_pc-1]:.3f})")
top_quality_features = quality_corr.head(3)
print(f"   - Top quality predictors:")
for feat, corr in top_quality_features.items():
    print(f"     * {feat}: r = {corr:.3f}")

print(f"\n5. ALCOHOL RELATIONSHIPS:")
print(f"   - PC{best_alcohol_pc} most correlated with alcohol (r = {pc_alcohol_corr[best_alcohol_pc-1]:.3f})")
print(f"   - Alcohol relates to density (inverse) and quality (positive)")

print(f"\n6. PRACTICAL IMPLICATIONS:")
print(f"   - Quality prediction: Focus on alcohol, volatile acidity, sulphates")
print(f"   - Feature engineering: Can create composite 'acidity index'")
print(f"   - Data collection: Some measurements (like density) may be redundant")
print(f"   - Model simplification: Use {n_90} PCs instead of {len(feature_cols)} features")

print("\n" + "="*80)
print("CONCLUSION: Eigenvalue analysis reveals that wine chemistry can be understood")
print("through a few dominant chemical signatures. Acidity and pH measurements are")
print("redundant (as expected chemically), and the 11 features can be effectively")
print(f"reduced to {n_90} principal components for most analyses.")
print("="*80)